## PACK THE WHOLE PROCESS INTO FUNCTIONS

In [1]:
from google.colab import drive
drive.mount('/content/drive')

root = '/content/drive/MyDrive/02_RETAIL'

Mounted at /content/drive


### Data Quality

In [2]:
def data_quality(x):

    #Modify types
    temp = x.astype({'month': 'O', 'wday': 'O'})

    #Imputar nulos
    temp.loc[x['event_name_1'].isna(),'event_name_1'] = 'no_event'

    def impute_mode(registers):
        #Calculates prpduct's price mode
        mode = registers.sell_price.mode()[0]
        #Impute nulls
        registers.loc[registers.sell_price.isna(),'sell_price'] = mode
        #Devuelve todos los registers del producto
        return(registers)

    temp = temp.groupby('item_id').apply(impute_mode)

    return(temp)


### Variable Creation

In [3]:
def create_variables(x):

    #INTERMITTENT DEMAND

    def stock_break(sales, n = 5):
        zero_sales = pd.Series(np.where(sales == 0,1,0))
        num_zeros = zero_sales.rolling(n).sum()
        stock_break = np.where(num_zeros == n,1,0)
        return(stock_break)

    x = x.sort_values(by = ['store_id','item_id','date'])
    x['stock_break_3'] = x.groupby(['store_id','item_id']).sales.transform(lambda x: stock_break(x, 3)).values
    x['stock_break_7'] = x.groupby(['store_id','item_id']).sales.transform(lambda x: stock_break(x,7)).values
    x['stock_break_15'] = x.groupby(['store_id','item_id']).sales.transform(lambda x: stock_break(x,15)).values


    #LAGS

    def create_lags(x, variable, num_lags = 7):
        lags = pd.DataFrame()
        for each in range(1,num_lags+1):
            lags[variable + '_lag_'+ str(each)] = x[variable].shift(each)
        return(lags)

    lags_sell_price_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: create_lags(x = x, variable = 'sell_price', num_lags= 7))

    lags_stock_break_3_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: create_lags(x = x, variable = 'stock_break_3', num_lags= 1))

    lags_stock_break_7_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: create_lags(x = x, variable = 'stock_break_7', num_lags= 1))

    lags_stock_break_15_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: create_lags(x = x, variable = 'stock_break_15', num_lags= 1))

    lags_sales_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: create_lags(x = x, variable = 'sales', num_lags= 15))


    #MOBILE WINDOWS

    def min_mobile(x, variable, num_periods = 7):
        minm = pd.DataFrame()
        for each in range(2,num_periods+1):
            minm[variable + '_minm_' + str(each)] = x[variable].shift(1).rolling(each).min()
        return(minm)

    def mean_mobile(x, variable, num_periods = 7):
        mm = pd.DataFrame()
        for each in range(2,num_periods+1):
            mm[variable + '_mm_' + str(each)] = x[variable].shift(1).rolling(each).mean()
        return(mm)

    def max_mobile(x, variable, num_periods = 7):
        maxm = pd.DataFrame()
        for each in range(2,num_periods+1):
            maxm[variable + '_maxm_' + str(each)] = x[variable].shift(1).rolling(each).max()
        return(maxm)

    min_mobile_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: min_mobile(x = x, variable = 'sales', num_periods= 15))

    mean_mobile_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: mean_mobile(x = x, variable = 'sales', num_periods= 15))

    max_mobile_x = x.groupby(['store_id','item_id'])\
                    .apply(lambda x: max_mobile(x = x, variable = 'sales', num_periods= 15))


    #JOIN DATAFRAMES

    x_joined = pd.concat([x,
                      lags_sell_price_x,
                      lags_stock_break_3_x,
                      lags_stock_break_7_x,
                      lags_stock_break_15_x,
                      lags_sales_x,
                      min_mobile_x,
                      mean_mobile_x,
                      max_mobile_x], axis = 1)

    x_joined.dropna(inplace=True)

    x_joined.drop(columns = ['sell_price','stock_break_3','stock_break_7','stock_break_15'],
                  inplace=True)

    #Create only one variable for the products
    x_joined.insert(loc=0,column='products',value=x_joined.store_id + '_'+ x_joined.item_id)
    x_joined = x_joined.drop(columns = ['store_id','item_id'])

    return(x_joined)

### Variable Transformation

In [4]:
def variable_transformation(x,y=None,mode = 'training'):

    '''
    This function to work for both training and execution:

     * Including the mode parameter, which defaults to training
     * Making the y parameter optional, since in execution it is not used

     When used in training mode it applies the fit_transform method and saves the objects.

     While when used in run mode it loads the objects and applies only the transform method.
    '''

    x.reset_index(inplace = True)

    #ENCODERS MANAGEMENT
    name_ohe = 'ohe_retail.pickle'
    name_te = 'te_retail.pickle'
    root_ohe = root + '/04_Models/' + name_ohe
    root_te = root + '/04_Models/' + name_te

    #ONE HOT ENCODING
    var_ohe = ['event_name_1']
    if mode == 'training':
        #If it's training it applies fit_transform and the encoder
        ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')
        ohe_x = ohe.fit_transform(x[var_ohe])
        ohe_x = pd.DataFrame(ohe_x, columns = ohe.get_feature_names_out())
        with open(root_ohe, mode='wb') as file:
           pickle.dump(ohe, file)
    else:
        #If it's in execution it retrieves it saves and applies only transform
        with open(root_ohe, mode='rb') as file:
            ohe = pickle.load(file)
        ohe_x = ohe.transform(x[var_ohe])
        ohe_x = pd.DataFrame(ohe_x, columns = ohe.get_feature_names_out())

    #TARGET ENCODING
    var_te = ['month','wday','weekday']
    if mode == 'training':
        #MAKE SURE IT HAS SAME REGISTERS AS X
        y.reset_index(inplace = True, drop = True)
        y = y.loc[y.index.isin(x.index)]
        #If it's training it applies fit_transform and the encoder
        te = TargetEncoder(min_samples_leaf=100, return_df = False)
        te_x = te.fit_transform(x[var_te], y = y)
        names_te = [variable + '_te' for variable in var_te]
        te_x = pd.DataFrame(te_x, columns = names_te)
        with open(root_te, mode='wb') as file:
           pickle.dump(te, file)
    else:
        #If it's in execution it retrieves it and applies only transform
        with open(root_te, mode='rb') as file:
            te = pickle.load(file)
        te_x = te.transform(x[var_te])
        names_te = [variable + '_te' for variable in var_te]
        te_x = pd.DataFrame(te_x, columns = names_te)


    #INTEGRATE, CLEANS AND RETURNS DF
    #Eliminates originals already transformed
    x = x.drop(columns=['event_name_1','month','wday','weekday'])
    #Incorporates the other dataframes
    x = pd.concat([x,ohe_x,te_x], axis=1).set_index('date')

    #Output
    return(x)

### Variable Preselection

In [5]:
def variable_preselection(x,y):

    '''
    Only use in training.
    '''
    #ELIMINATES THE products COLUMN AND INDEX
    x.reset_index(drop = True,inplace = True)
    x.drop(columns='products',inplace = True)

    #MAKES SURE IT HAS THE SAME REGISTERS AS X
    y = y.loc[y.index.isin(x.index)]


    mutual_selector = mutual_info_regression(x,y)
    variable_limit_position = 70
    ranking_mi = pd.DataFrame(mutual_selector, index = x.columns).reset_index()
    ranking_mi.columns = ['variable','importance_mi']
    ranking_mi = ranking_mi.sort_values(by = 'importance_mi', ascending = False)
    ranking_mi['ranking_mi'] = np.arange(0,ranking_mi.shape[0])
    goes_in_mi = ranking_mi.iloc[0:variable_limit_position].variable
    x_mi = x[goes_in_mi].copy()

    return(x_mi)

### Model

#### Training

In [6]:
def modelize(x_products, y):

    '''
    This function does the individual modeling.

    Receives the x and y data of a product.

    Find the optimal parameters for that product.

    Returns the best model.
    '''

    #Exclude product as a modeling variable
    var_modelize = x_products.columns.to_list()[2:]

    #Defines cross validation
    time_cv = TimeSeriesSplit(5, test_size = 8)

    #Defines algorithms grid
    pipe = Pipeline([('algorithm',HistGradientBoostingRegressor())])

    grid = [
         {'algorithm': [HistGradientBoostingRegressor()]
#          'algorithm__learning_rate': [0.01,0.025,0.05,0.1],
#          'algorithm__max_iter': [50,100,200],
#          'algorithm__max_depth': [5,10,20,50],
#          'algorithm__scoring': ['neg_mean_absolute_error'],
#          'algorithm__l2_regularization': [0,0.25,0.5,0.75,1]
         }

    ]

    #Create the models
    random_search = RandomizedSearchCV(estimator = pipe,
                                   param_distributions = grid,
                                   n_iter = 1,
                                   cv = time_cv,
                                   scoring = 'neg_mean_absolute_error',
                                   verbose = 0,
                                   n_jobs = -1)

    model = random_search.fit(x_products[var_modelize],y)

    #Retrain the best over all data
    final_model = model.best_estimator_.fit(x_products[var_modelize],y)

    #Returns the final model as output
    return(final_model)

In [7]:
def launch_training(df):

    '''
    This function goes through all the productss and calls modelize() to create a total list with all the models of all the productss.

     It receives the dataframe of the x's already cleaned and segmented by products, and also the target.

     It does not return anything, but saves the object already trained with all the models on disk.
    '''

    products_list = list(df.products.unique())

    models_list =[]

    for each in products_list:

        #Rename so we have clarity
        products = each
        target = 'sales'

        x = df.loc[df.products == products].copy().drop(columns=target).copy()
        y = df.loc[df.products == products,'sales'].copy()

        x = variable_transformation(x,y)
        x = variable_preselection(x,y)

        #Calls modelize
        model = modelize(x,y)

        #Adds the final model to the list
        models_list.append((products,model))

    #Save the train models list
    name_models = 'models_list_retail.pickle'
    root_models = root + '/04_Models/' + name_models
    with open(root_models, mode='wb') as file:
       pickle.dump(models_list, file)

#### Execution

In [8]:
#Launch prediction
def launch_prediction(df):

    '''
    This function makes the forecast for each products, but only for one day.

    Receives the new dataset to predict.

    It must have the structure of the file DataForProduction.csv in the Validation folder.

    It goes through each products, loading its corresponding model, selecting its data, and making predictions.

    Returns the prediction for all the productss but ONLY FOR THE CORRESPONDING DAY.
    '''

    #LOAD MODELS
    name_models = 'models_list_retail.pickle'
    root_models = root + '/04_Models/' + name_models
    with open(root_models, mode='rb') as file:
       models_list = pickle.load(file)

    df_predictions = pd.DataFrame(columns=['date','products','sales','prediction'])

    for each in range(0,len(models_list)):

        products = models_list[each][0]
        model = models_list[each][1]
        variables = model[0].feature_names_in_
        target = 'sales'

        x = df.loc[df.products == products].copy().drop(columns=target).copy()
        y = df.loc[df.products == products,'sales'].copy()

        date = df.reset_index().copy()
        date = date.loc[date.products == products,'date'].values

       #Variable transformation
        x = variable_transformation(x, mode = 'execution')

        #Variable selection
        x = x[variables]

        #Predictions calculations
        predictions = pd.DataFrame(data={'date': date,
                                          'products': products,
                                          'sales': y,
                                          'prediction': model.predict(x)})

        predictions['prediction'] = predictions.prediction.astype('int')

        df_predictions = pd.concat([df_predictions,predictions])

    df_predictions = df_predictions.loc[df_predictions.index == df_predictions.index.min()]
    return(df_predictions)

In [9]:
def forecast_recursive(x):

    '''
    This function is the one that applies the recursive forecast to predict 8 days.

     Receives the new dataset to predict.

     It must have the structure of the file DataForproductsion.csv in the Validation folder.

     To apply recursion:

     * It will predict the first day for which it has all the information (ie 15 days from the oldest day)
     * When finished, it saves the sales prediction in the record to be predicted and eliminates the records of the oldest day in the data frame.
     * Therefore in the next iteration it will predict the next day.

     For example:

     If the oldest day in the dataset is 12/09/2015 then the first day you can predict

     (and of which we no longer have data) is 12/24/2015.

     When you predict the data of 24 for each products you overwrite it as your sales

     and removes all records from day 09.

     Then the oldest day becomes the 10th and therefore the day to predict is the 25th.

     And so until the end of 8 cycles to predict the week we want.
    '''

    for each in range(0,8):
        df_step1 = data_quality(x)
        df_step2 = create_variables(df_step1)

        #Calculates prediction
        f = launch_prediction(df_step2)
        f['store_id'] = f.products.str[:4]
        f['item_id'] = f.products.str[5:]

        #Updates the sales data with the prediction
        x.loc[(x.index.isin(f.date)) & (x.store_id.isin(f.store_id)) & (x.item_id.isin(f.item_id)),'sales'] = f.prediction

        #Eliminates the oldest day from x
        x = x.loc[x.index != x.index.min()]

    return(x)

## PROCESS

### RETRAIN

In [10]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.0 MB/s eta 0:00:00


In [11]:
import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

from sklearn.feature_selection import mutual_info_regression

from sklearn.model_selection import TimeSeriesSplit

from sklearn.pipeline import Pipeline

from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

#Load Data
# ruta_proyecto = 'C:/Users/isaac/Google Drive/DS4B/CursoMachineLearningPython/03_MACHINE_LEARNING/08_CASOS/02_RETAIL'
data_file_name = 'work.csv'
root_complete = root + '/02_Data/03_Work/' + data_file_name
df = pd.read_csv(root_complete,sep=',',parse_dates=['date'],index_col='date')

#Seldct only the used ones
final_variables = ['store_id',
                     'item_id',
                     'event_name_1',
                     'month',
                     'sell_price',
                     'wday',
                     'weekday',
                     'sales']

df = df[final_variables]

df_step1 = data_quality(df)
df_step2 = create_variables(df_step1)

launch_training(df_step2)

### EVALUATION

In [12]:
import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

from sklearn.feature_selection import mutual_info_regression

from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

#Load data
# root = 'C:/Users/isaac/Google Drive/DS4B/CursoMachineLearningPython/03_MACHINE_LEARNING/08_CASOS/02_RETAIL'
data_file_name = 'validation.csv'
root_complete = root + '/02_Data/02_Validation/' + data_file_name
df = pd.read_csv(root_complete,sep=',',parse_dates=['date'],index_col='date')

#Seldct only the used ones
final_variables = ['store_id',
                     'item_id',
                     'event_name_1',
                     'month',
                     'sell_price',
                     'wday',
                     'weekday',
                     'sales']


df = df[final_variables]

df_step1 = data_quality(df)
df_step2 = create_variables(df_step1)

forecast_1day = launch_prediction(df_step2)

print('MAE = ', mean_absolute_error(forecast_1day.sales,forecast_1day.prediction))

forecast_1day

MAE =  5.45


,date,products,sales,prediction
2015-12-16,2015-12-16,CA_3_FOODS_3_090,0,-9
2015-12-16,2015-12-16,CA_3_FOODS_3_120,52,49
2015-12-16,2015-12-16,CA_3_FOODS_3_202,20,13
2015-12-16,2015-12-16,CA_3_FOODS_3_252,36,34
2015-12-16,2015-12-16,CA_3_FOODS_3_288,35,22
2015-12-16,2015-12-16,CA_3_FOODS_3_329,64,43
2015-12-16,2015-12-16,CA_3_FOODS_3_555,30,28
2015-12-16,2015-12-16,CA_3_FOODS_3_586,76,62
2015-12-16,2015-12-16,CA_3_FOODS_3_587,29,31
2015-12-16,2015-12-16,CA_3_FOODS_3_714,19,17


### EXECUTION

In [25]:
import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

from sklearn.feature_selection import mutual_info_regression

from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

#Load Data
root = '/content/drive/MyDrive/02_RETAIL'
data_file_name = 'DataForProduction.csv'
root_complete = root + '/02_Data/02_Validation/' + data_file_name
df = pd.read_csv(root_complete,parse_dates=['date'],index_col='date')

#Select only the used ones
final_variables = ['store_id',
                     'item_id',
                     'event_name_1',
                     'month',
                     'sell_price',
                     'wday',
                     'weekday',
                     'sales']

df = df[final_variables]

#Launch prediction
forecast = forecast_recursive(df)

forecast.sort_values(by = ['store_id','item_id'])

,store_id,item_id,event_name_1,month,sell_price,wday,weekday,sales
date,,,,,,,,
2015-12-17,CA_3,FOODS_3_090,NaN,12,1.00,6,Thursday,0
2015-12-18,CA_3,FOODS_3_090,NaN,12,1.00,7,Friday,1
2015-12-19,CA_3,FOODS_3_090,NaN,12,1.00,1,Saturday,0
2015-12-20,CA_3,FOODS_3_090,NaN,12,1.00,2,Sunday,6
2015-12-21,CA_3,FOODS_3_090,NaN,12,1.00,3,Monday,4
...,...,...,...,...,...,...,...,...
2015-12-27,CA_4,FOODS_3_714,NaN,12,1.58,2,Sunday,10
2015-12-28,CA_4,FOODS_3_714,NaN,12,1.58,3,Monday,7
2015-12-29,CA_4,FOODS_3_714,NaN,12,1.58,4,Tuesday,6
